Test

In [5]:

import pandas as pd             # manipulation de données tabulaires
import sys
import os
import json                     # pour afficher proprement le JSON si besoin

sys.path.append(os.path.abspath(".."))    # remonter d’un cran depuis notebooks/
sys.path.append(os.path.abspath("../src"))


from src.etl import fetch_matches, classement_interactif #,pts, classement_interactif
# Cellule: obtenir le classement (standings) d'une compétition
competition_id = 2015  # exemple: 2015 = Ligue 1 (selon football-data.org), 2021 = Premier League
# Appel de l'API pour récupérer les standings
matches_json = fetch_matches(2015,2025)


standings_long=classement_interactif(matches_json)
standings_long.head()

,matchday,team,points_cum,gf_cum,ga_cum,gdif_cum,wins_cum,draws_cum,losses_cum,rank
2,1,AS Monaco FC,3,3.0,1.0,2.0,1,0,0,1
5,1,AJ Auxerre,3,1.0,0.0,1.0,1,0,0,2
7,1,Angers SCO,3,1.0,0.0,1.0,1,0,0,3
109,1,Olympique Lyonnais,3,1.0,0.0,1.0,1,0,0,4
116,1,Paris Saint-Germain FC,3,1.0,0.0,1.0,1,0,0,5
